In [0]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

# If you get an error running this cell, then please head over to the troubleshooting notebook!

# And now the change for Ollama

1. No environment variables are needed (no keys) so this part has been removed

2. The OpenAI client library is being initialized to point to your local computer for Ollama

3. You need to have installed Ollama on your computer, and run `ollama run llama3.2` in a Powershell or Terminal if you haven't already

4. Anywhere in this lab that it used to have **gpt-4o-mini** it now has **lama3.2**


In [0]:
# Here it is - see the base_url

openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')


# Let's make a quick call to a Frontier model to get started, as a preview!

In [0]:
# To give you a preview -- calling OpenAI with these messages is this easy. Any problems, head over to the Troubleshooting notebook.

message = "Hello, Llama! This is my first ever message to you! Hi!"
response = openai.chat.completions.create(model="llama3.2", messages=[{"role":"user", "content":message}])
print(response.choices[0].message.content)

## OK onwards with our first project

In [0]:
# A class to represent a Webpage
# If you're not familiar with Classes, check out the "Intermediate Python" notebook

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [0]:
# Let's try one out. Change the website and add print statements to follow along.

ed = Website("https://sohanpatharla.vercel.app/about")
print(ed.title)
print("Title is printed above")
print(ed.text)

In [0]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [0]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

## And now let's build useful messages for GPT-4o-mini, using a function

In [0]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

## Time to bring it together - the API for OpenAI is very simple!

In [0]:
# And now: call the OpenAI API. You will get very familiar with this!

def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "llama3.2",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [0]:
# A function to display this nicely in the Jupyter output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [0]:
display_summary("https://sohanpatharla.vercel.app/about")

# Let's try more websites

Note that this will only work on websites that can be scraped using this simplistic approach.

Websites that are rendered with Javascript, like React apps, won't show up. See the community-contributions folder for a Selenium implementation that gets around this. You'll need to read up on installing Selenium (ask ChatGPT!)

Also Websites protected with CloudFront (and similar) may give 403 errors - many thanks Andy J for pointing this out.

But many websites will work just fine!

In [0]:
display_summary("https://openai.com")

In [0]:
display_summary("https://anthropic.com")

## Email Subject Suggestion based on the letter body

In [0]:
# Step 1: Create your prompts

system_prompt = """You are an assistant that analyzes the contents of an email letter body \
and provide a appropriate short subject line for that email,based on that email body. \
"""
user_prompt = """
    \nThe contents of an email body is as follows; \
understand the content in that well and provide me a appropriate subject based on the text content in it. \
Understand the sentiment of the email and choose the subject type to be formal or informal or anything.\n\n
"""

# Step 2: Make the messages list

messages = [
    {"role": "system", "content": system_prompt},
    
    {"role": "user", "content": user_prompt + """
Hey John, just wanted to say thanks for your help with the move last weekend! Couldn't have done it without you.
"""},

    {"role": "user", "content": user_prompt + """
Dear Hiring Manager, I am writing to express my interest in the Marketing Manager position listed on your company’s website.
"""},

    {"role": "user", "content": user_prompt + """
We are excited to invite you to our annual developer conference taking place in San Francisco this July. Register today to secure your spot!
"""},

    {"role": "user", "content": user_prompt + """
Hello, I'm following up on the support ticket I submitted last week regarding the issue with logging into my account. I still haven’t received a resolution.
"""},

    {"role": "user", "content": user_prompt + """
Congratulations! You've been selected as one of our winners in the Spring Giveaway Contest. Claim your prize by replying to this email.
"""},

    {"role": "user", "content": user_prompt + """
Good morning team, just a reminder that our Q2 strategy meeting is scheduled for 10 AM tomorrow in Conference Room B.
"""},

    {"role": "user", "content": user_prompt + """
Hi Mom, the flight was fine, and I got here safely. The weather’s great and the Airbnb is cozy. I’ll send pictures soon!
"""},

    {"role": "user", "content": user_prompt + """
To whom it may concern, I am very dissatisfied with the quality of the product I received and would like a full refund.
"""}
]


# Step 3: Call OpenAI

response =openai.chat.completions.create(model="llama3.2",messages=messages)

# Step 4: print the result
# response = openai.chat.completions.create(model="llama3.2", messages=messages)
#print(response.choices[0].message.content)
print(response.choices[0].message.content)

## An extra exercise for those who enjoy web scraping

You may notice that if you try `display_summary("https://openai.com")` - it doesn't work! That's because OpenAI has a fancy website that uses Javascript. There are many ways around this that some of you might be familiar with. For example, Selenium is a hugely popular framework that runs a browser behind the scenes, renders the page, and allows you to query it. If you have experience with Selenium, Playwright or similar, then feel free to improve the Website class to use them. In the community-contributions folder, you'll find an example Selenium solution from a student (thank you!)

In [0]:
!pip install selenium

In [0]:
#Parse webpages which is designed using JavaScript heavely
# download the chorme driver from here as per your version of chrome - https://developer.chrome.com/docs/chromedriver/downloads
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

PATH_TO_CHROME_DRIVER = r'C:\Users\sohan\Downloads\chromedriver-win64\chromedriver-win64\chromedriver.exe'

class Website:
    url: str
    title: str
    text: str

    def __init__(self, url):
        self.url = url

        options = Options()

        options.add_argument("--no-sandbox")
        options.add_argument("--disable-dev-shm-usage")

        service = Service(PATH_TO_CHROME_DRIVER)
        driver = webdriver.Chrome(service=service, options=options)
        driver.get(url)

        input("Please complete the verification in the browser and press Enter to continue...")
        page_source = driver.page_source
        driver.quit()

        soup = BeautifulSoup(page_source, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.get_text(separator="\n", strip=True)

In [0]:
display_summary("https://openai.com")